In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

### Importazione dataframe

In [9]:
df1 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_23mar_to_29mar.csv')
df2 = pd.read_csv('C:/Users/HP/Desktop/PLC_gualchierani/monitorconi_30mar_to_4apr.csv')

In [10]:
df1["measured_at"] = pd.to_datetime( df1["measured_at"] )
df2["measured_at"] = pd.to_datetime( df2["measured_at"] )

In [11]:
mask_30mar = np.asarray( df1[ df1['measured_at'].dt.day == 30].index )
df1.drop( mask_30mar, inplace = True )

mask_29mar = np.asarray( df2[ df2['measured_at'].dt.day == 29].index )
df2.drop( mask_29mar, inplace = True )

mask_5apr = np.asarray( df2[ df2['measured_at'].dt.day == 5].index )
df2.drop( mask_5apr, inplace = True )

In [14]:
df = pd.concat([df1,df2])

In [16]:
df.drop(['id', 'tipologia_lavorazione', 'cicli_da_eseguire',
        'cicli_eseguiti', 'tempo_miscelazione_eseguito',
        'stato_tramoggia', 'tara_cono', 'lotto',
        'created_at', 'updated_at'], axis=1, inplace=True)

In [17]:
mask_no_cono = np.asarray( df[ df['id_cono'] > 24 ].index )
df.drop( mask_no_cono, inplace = True )

In [18]:
len(df)

652515

### Preparazione dati di supporto

In [19]:
buffer = [n for n in range(21, 29)]
buffer.extend([n for n in range(31, 39)])

legenda_gualchierani = {
    'fuori_impianto': [0],
    'ing_zona_misc': [1],
    'catenaria_ing_misc': [45, 46],
    'navetta_serv_misc': [43],
    'pesatura': [41, 42],
    'miscelatore': [2, 8],
    'cat_usc_pieni': [3],
    'nav_serv_carr_pieni': [4],
    'nav_serv_carr_vuoti': [5],
    'buffer': buffer,
    'carroponte_1': [6],
    'carroponte_2': [7],
    'estrusore': [n for n in range(11, 19)],
    'handling_post_est': [6, 5, 4],
    'catenaria_vuoti': [47, 48],
    'catenaria_uscita_vuoti': [49],
    'pinza_vuoti': [50],
    'catenaria_parcheggi': [n for n in range(51, 58)],
    'nav_serv_parcheggi': [44],
    'manuale': [68, 70]
}

dict_states = {
    0: 'Not_def',
    1: 'V_per_park',
    2: 'P_da_misc',
    3: 'P_da_est',
    4: 'P_remove',
    5: 'P_recheck',
    6: 'P_per_park',
    7: 'V_da_pes',
    8: 'V_remove',
    9: 'V_lavaggio'
}

starter = ['V_da_pes/nav_serv_carr_pieni', 'V_da_pes/nav_serv_carr_vuoti',
'V_da_pes/catenaria_vuoti', 'V_da_pes/catenaria_parcheggi',
'V_da_pes/nav_serv_parcheggi', 'V_da_pes/catenaria_ing_misc',
'V_da_pes/ing_zona_misc']

finisher = ['P_da_est/estrusore', 'Not_def/fuori_impianto']

finisher2 = ['V_per_park/estrusore', 'V_da_pes/estrusore', 'Not_def/fuori_impianto']

### Preparazione funzioni

In [20]:
#  questa funzione estrae dal df il primo record in ordine temporale per ogni ubicazione

def create_df_t(df, cono_in_esame):
    mask = df['id_cono'] == cono_in_esame
    df1 = df[mask]

    ubicazione = []
    stato = []
    timestamp = []
    for i in range(len(df1) - 1):
        if i == 0:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])
        elif df1.iloc[i, 1] == df1.iloc[i - 1, 1]:
            continue
        else:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])

#  estrae i valori dalla memoria del PLC li traduco in parole
    for i in range(len(ubicazione)):
        for k,v in legenda_gualchierani.items():
            if ubicazione[i] in v:
                ubicazione[i] = k

    for i in range(len(stato)):
        stato[i] = dict_states[stato[i]]

#  crea il dataframe che sarà input alla funzione di individuazione della tracce
    df_t = pd.DataFrame(list(zip(stato, ubicazione, timestamp)), columns =['stato', 'ubicazione', 'timestamp'])
    df_t.sort_values('timestamp', inplace=True)
    df_t['ID'] = 0
    df_t['CRC'] = 0
    df_t['rif'] = (df_t.stato.astype('str') + '/' + df_t.ubicazione.astype('str'))
    return(df_t)

In [21]:
#  questa funzione estrae dal df il primo record in ordine temporale per ogni ubicazione

def create_df_t2(df, cono_in_esame):
    mask = df['id_cono'] == cono_in_esame
    df1 = df[mask]

    ubicazione = []
    stato = []
    timestamp = []
    for i in range(len(df1) - 1):
        if i == 0:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])
        if df1.iloc[i, 1] in legenda_gualchierani['estrusore']:
            if ((df1.iloc[i, 1] == df1.iloc[i - 1, 1] and df1.iloc[i, 1] != df1.iloc[i + 1, 1])
                    or df1.iloc[i, 1] != df1.iloc[i - 1, 1]):
                ubicazione.append(df1.iloc[i, 1])
                stato.append(df1.iloc[i, 2])
                timestamp.append(df1.iloc[i, 8])
        elif df1.iloc[i, 1] == df1.iloc[i - 1, 1]:
            continue
        else:
            ubicazione.append(df1.iloc[i, 1])
            stato.append(df1.iloc[i, 2])
            timestamp.append(df1.iloc[i, 8])

#  estrae i valori dalla memoria del PLC li traduco in parole
    for i in range(len(ubicazione)):
        for k,v in legenda_gualchierani.items():
            if ubicazione[i] in v:
                ubicazione[i] = k

    for i in range(len(stato)):
        stato[i] = dict_states[stato[i]]

#  crea il dataframe che sarà input alla funzione di individuazione della tracce
    df_t = pd.DataFrame(list(zip(stato, ubicazione, timestamp)), columns =['stato', 'ubicazione', 'timestamp'])
    df_t.sort_values('timestamp', inplace=True)
    df_t['ID'] = 0
    df_t['CRC'] = 0
    df_t['rif'] = (df_t.stato.astype('str') + '/' + df_t.ubicazione.astype('str'))
    df_t.drop_duplicates(inplace=True)
    return(df_t)

In [22]:
#  questa funzione individua delle potenziali tracce di processo
#  prende in input una lista di possibili punti di inizio ed una di possibili punti di fine

def create_traces(df_t, cono_in_esame, starter, finisher):
    global trace
    df_t2 = df_t.copy()
    cont_related_counter = 0
    bol_end = False
    bol_start = False
    for idx in df_t2.index:
        global trace
        if df_t2.loc[idx, 'rif'] in starter and not bol_start:
            bol_end = False
            bol_start = True
            trace += 1
            cont_related_counter += 1
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
        elif df_t2.loc[idx, 'rif'] == 'V_da_pes/pesatura' and not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
            bol_end = True
            bol_start = False
        elif not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
    
    #  elimina dal df tutti i record che non sono stati attribuiti a nessuna traccia
    mask_no_trace = np.asarray( df_t2[ df_t2['ID'] == 0 ].index )
    df_t2.drop( mask_no_trace, inplace = True )
    
    #  qualche trick per limitare i doppioni
    trace = trace + 1
    df_t2['ID_unique'] = (df_t2.CRC.astype('str') + 'C' + str(cono_in_esame) + df_t2.ID.astype('str'))
    return(df_t2)

In [23]:
def create_traces3(df_t, cono_in_esame, finisher):
    global trace
    df_t2 = df_t.copy()
    cont_related_counter = 0
    bol_end = False
    bol_start = False
    for idx in df_t2.index:
        global trace
        if df_t2.loc[idx, 'rif'] == 'V_da_pes/pesatura' and not bol_start:
            bol_end = False
            bol_start = True
            trace += 1
            cont_related_counter += 1
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
        elif df_t2.loc[idx, 'rif'] in finisher and not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
            bol_end = True
            bol_start = False
        elif not bol_end:
            df_t2.loc[idx, 'ID'] = trace
            df_t2.loc[idx, 'CRC'] = cont_related_counter
    
    mask_no_trace = np.asarray( df_t2[ df_t2['ID'] == 0 ].index )
    df_t2.drop( mask_no_trace, inplace = True )
    
    #  qualche trick per limitare i doppioni
    trace = trace + 10
    df_t2['ID_unique'] = (df_t2.CRC.astype('str') + 'C' + str(cono_in_esame) + df_t2.ID.astype('str'))
    if df_t2['CRC'].isnull().values.any():
        print('CI SONO DEI NULL SECONDA FUNZIONE  CONO  ', cono_in_esame)
    return(df_t2)

### Calcolo tracce

In [24]:
trace = 0
for c in range(1, 25):
    global trace
    start = datetime.now()
    df_pre_trace = create_df_t2(df, c)
    if c == 1:
        df_traces = create_traces3(df_pre_trace, c, finisher2)
    else:
        df_traces = pd.concat([df_traces, create_traces3(df_pre_trace, c, finisher2)])
    end = datetime.now()
    print('TEMPO ELABORAZIONE CONO {} {}'.format(c, end-start))

TEMPO ELABORAZIONE CONO 1 0:00:07.607001
TEMPO ELABORAZIONE CONO 2 0:00:06.780072
TEMPO ELABORAZIONE CONO 3 0:00:09.873395
TEMPO ELABORAZIONE CONO 4 0:00:08.749080
TEMPO ELABORAZIONE CONO 5 0:00:08.416402
TEMPO ELABORAZIONE CONO 6 0:00:06.596783
TEMPO ELABORAZIONE CONO 7 0:00:07.337372
TEMPO ELABORAZIONE CONO 8 0:00:15.228656
TEMPO ELABORAZIONE CONO 9 0:00:08.271386
TEMPO ELABORAZIONE CONO 10 0:00:10.851310
TEMPO ELABORAZIONE CONO 11 0:00:08.858711
TEMPO ELABORAZIONE CONO 12 0:00:07.338145
TEMPO ELABORAZIONE CONO 13 0:00:15.182576
TEMPO ELABORAZIONE CONO 14 0:00:07.835445
TEMPO ELABORAZIONE CONO 15 0:00:08.326625
TEMPO ELABORAZIONE CONO 16 0:00:06.821199
TEMPO ELABORAZIONE CONO 17 0:00:11.068619
TEMPO ELABORAZIONE CONO 18 0:00:08.601167
TEMPO ELABORAZIONE CONO 19 0:00:07.595840
TEMPO ELABORAZIONE CONO 20 0:00:06.052326
TEMPO ELABORAZIONE CONO 21 0:00:07.610213
TEMPO ELABORAZIONE CONO 22 0:00:11.083341
TEMPO ELABORAZIONE CONO 23 0:00:14.913696
TEMPO ELABORAZIONE CONO 24 0:00:08.272736


In [28]:
df_traces.to_csv('C:/Users/HP/Desktop/df_traces_from_weigh.csv')

In [26]:
len(df_traces['ID_unique'].unique())

1193

In [27]:
df_traces

,stato,ubicazione,timestamp,ID,CRC,rif,ID_unique
18,V_da_pes,pesatura,2022-03-23 01:54:52,1,1,V_da_pes/pesatura,1C11
19,P_da_misc,navetta_serv_misc,2022-03-23 02:10:53,1,1,P_da_misc/navetta_serv_misc,1C11
20,P_da_misc,miscelatore,2022-03-23 02:11:28,1,1,P_da_misc/miscelatore,1C11
21,P_da_est,cat_usc_pieni,2022-03-23 02:14:54,1,1,P_da_est/cat_usc_pieni,1C11
22,P_da_est,nav_serv_carr_vuoti,2022-03-23 02:17:45,1,1,P_da_est/nav_serv_carr_vuoti,1C11
...,...,...,...,...,...,...,...
2217,P_da_est,carroponte_1,2022-04-04 23:21:00,1392,126,P_da_est/carroponte_1,126C231392
2218,P_da_est,buffer,2022-04-04 23:23:17,1392,126,P_da_est/buffer,126C231392
2219,P_da_est,carroponte_2,2022-04-04 23:26:09,1392,126,P_da_est/carroponte_2,126C231392
2220,P_da_est,estrusore,2022-04-04 23:26:43,1392,126,P_da_est/estrusore,126C231392


In [15]:
6650
580

580